# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 19 2022 12:06PM,253373,22,636504,"NBTY Global, Inc.",Released
1,Dec 19 2022 11:53AM,253371,10,8921191,"Snap Medical Industries, LLC",Released
2,Dec 19 2022 11:53AM,253371,10,8921192,"Snap Medical Industries, LLC",Released
3,Dec 19 2022 11:53AM,253370,10,8922261,"Snap Medical Industries, LLC",Released
4,Dec 19 2022 11:53AM,253370,10,8922262,"Snap Medical Industries, LLC",Released
5,Dec 19 2022 11:53AM,253370,10,8922263,"Snap Medical Industries, LLC",Released
6,Dec 19 2022 11:53AM,253370,10,8922267,"Snap Medical Industries, LLC",Released
7,Dec 19 2022 11:45AM,253369,10,Stone-8922327,Acute Outpatient Solutions,Released
8,Dec 19 2022 11:43AM,253368,10,PRF-45122,Bio-PRF,Released
9,Dec 19 2022 11:37AM,253367,19,60018270,"GUSA Granules USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,253368,Released,1
21,253369,Released,1
22,253370,Released,4
23,253371,Released,2
24,253373,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253368,NaN,NaN,1.0
253369,NaN,NaN,1.0
253370,NaN,NaN,4.0
253371,NaN,NaN,2.0
253373,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253332,1.0,2.0,10.0
253340,0.0,13.0,6.0
253341,0.0,25.0,14.0
253344,0.0,0.0,6.0
253346,0.0,33.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253332,1,2,10
253340,0,13,6
253341,0,25,14
253344,0,0,6
253346,0,33,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253332,1,2,10
1,253340,0,13,6
2,253341,0,25,14
3,253344,0,0,6
4,253346,0,33,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253332,1,2,10
1,253340,,13,6
2,253341,,25,14
3,253344,,,6
4,253346,,33,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 19 2022 12:06PM,253373,22,"NBTY Global, Inc."
1,Dec 19 2022 11:53AM,253371,10,"Snap Medical Industries, LLC"
3,Dec 19 2022 11:53AM,253370,10,"Snap Medical Industries, LLC"
7,Dec 19 2022 11:45AM,253369,10,Acute Outpatient Solutions
8,Dec 19 2022 11:43AM,253368,10,Bio-PRF
9,Dec 19 2022 11:37AM,253367,19,"GUSA Granules USA, Inc."
10,Dec 19 2022 11:31AM,253366,10,ISDIN Corporation
15,Dec 19 2022 11:05AM,253364,10,"Senseonics, Incorporated"
16,Dec 19 2022 10:44AM,253362,19,"GUSA Granules USA, Inc."
17,Dec 19 2022 10:19AM,253358,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 19 2022 12:06PM,253373,22,"NBTY Global, Inc.",,,1
1,Dec 19 2022 11:53AM,253371,10,"Snap Medical Industries, LLC",,,2
2,Dec 19 2022 11:53AM,253370,10,"Snap Medical Industries, LLC",,,4
3,Dec 19 2022 11:45AM,253369,10,Acute Outpatient Solutions,,,1
4,Dec 19 2022 11:43AM,253368,10,Bio-PRF,,,1
5,Dec 19 2022 11:37AM,253367,19,"GUSA Granules USA, Inc.",,,1
6,Dec 19 2022 11:31AM,253366,10,ISDIN Corporation,,,5
7,Dec 19 2022 11:05AM,253364,10,"Senseonics, Incorporated",,,1
8,Dec 19 2022 10:44AM,253362,19,"GUSA Granules USA, Inc.",,,1
9,Dec 19 2022 10:19AM,253358,15,"Brookfield Pharmaceuticals, LLC",,,22


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 19 2022 12:06PM,253373,22,"NBTY Global, Inc.",1,,
1,Dec 19 2022 11:53AM,253371,10,"Snap Medical Industries, LLC",2,,
2,Dec 19 2022 11:53AM,253370,10,"Snap Medical Industries, LLC",4,,
3,Dec 19 2022 11:45AM,253369,10,Acute Outpatient Solutions,1,,
4,Dec 19 2022 11:43AM,253368,10,Bio-PRF,1,,
5,Dec 19 2022 11:37AM,253367,19,"GUSA Granules USA, Inc.",1,,
6,Dec 19 2022 11:31AM,253366,10,ISDIN Corporation,5,,
7,Dec 19 2022 11:05AM,253364,10,"Senseonics, Incorporated",1,,
8,Dec 19 2022 10:44AM,253362,19,"GUSA Granules USA, Inc.",1,,
9,Dec 19 2022 10:19AM,253358,15,"Brookfield Pharmaceuticals, LLC",22,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 19 2022 12:06PM,253373,22,"NBTY Global, Inc.",1,,
1,Dec 19 2022 11:53AM,253371,10,"Snap Medical Industries, LLC",2,,
2,Dec 19 2022 11:53AM,253370,10,"Snap Medical Industries, LLC",4,,
3,Dec 19 2022 11:45AM,253369,10,Acute Outpatient Solutions,1,,
4,Dec 19 2022 11:43AM,253368,10,Bio-PRF,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 19 2022 12:06PM,253373,22,"NBTY Global, Inc.",1.0,NaN,NaN
1,Dec 19 2022 11:53AM,253371,10,"Snap Medical Industries, LLC",2.0,NaN,NaN
2,Dec 19 2022 11:53AM,253370,10,"Snap Medical Industries, LLC",4.0,NaN,NaN
3,Dec 19 2022 11:45AM,253369,10,Acute Outpatient Solutions,1.0,NaN,NaN
4,Dec 19 2022 11:43AM,253368,10,Bio-PRF,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 19 2022 12:06PM,253373,22,"NBTY Global, Inc.",1.0,0.0,0.0
1,Dec 19 2022 11:53AM,253371,10,"Snap Medical Industries, LLC",2.0,0.0,0.0
2,Dec 19 2022 11:53AM,253370,10,"Snap Medical Industries, LLC",4.0,0.0,0.0
3,Dec 19 2022 11:45AM,253369,10,Acute Outpatient Solutions,1.0,0.0,0.0
4,Dec 19 2022 11:43AM,253368,10,Bio-PRF,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3293632,191.0,78.0,0.0
15,760063,47.0,33.0,0.0
19,760061,12.0,2.0,1.0
22,253373,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3293632,191.0,78.0,0.0
1,15,760063,47.0,33.0,0.0
2,19,760061,12.0,2.0,1.0
3,22,253373,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,191.0,78.0,0.0
1,15,47.0,33.0,0.0
2,19,12.0,2.0,1.0
3,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,191.0
1,15,Released,47.0
2,19,Released,12.0
3,22,Released,1.0
4,10,Executing,78.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,22
Status,,,,
Completed,0.0,0.0,1.0,0.0
Executing,78.0,33.0,2.0,0.0
Released,191.0,47.0,12.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,22
0,Completed,0.0,0.0,1.0,0.0
1,Executing,78.0,33.0,2.0,0.0
2,Released,191.0,47.0,12.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,22
0,Completed,0.0,0.0,1.0,0.0
1,Executing,78.0,33.0,2.0,0.0
2,Released,191.0,47.0,12.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()